In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.callbacks import get_openai_callback

set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

examples = [
    {
        "movie": "Shrek",
        "answer": """
        Here is what I know:
        Director: Andrew Adamson, Vicky Jenson
        Cast: Mike Myers (Shrek), Eddie Murphy (Donkey), Cameron Diaz (Princess Fiona), John Lithgow (Lord Farquaad)
        Budget: $60 million
        Box office revenue: $487.9 million
        Genre: Animation, Comedy, Adventure, Fantasy
        Brief Synopsis: A grumpy but kind-hearted ogre named Shrek embarks on a journey to rescue Princess Fiona, encountering a talkative donkey and uncovering unexpected secrets along the way.
        """,
    },
    {
        "movie": "Top Gun",
        "answer": """
        I know this:
        Director: Tony Scott
        Cast: Tom Cruise (Maverick), Kelly McGillis (Charlie), Val Kilmer (Iceman), Anthony Edwards (Goose)
        Budget: $15 million
        Box office revenue: $357.3 million
        Genre: Action, Drama
        Brief Synopsis: A talented but reckless Navy pilot, Pete "Maverick" Mitchell, trains at the elite Top Gun Naval Fighter Weapons School, facing rivalries, romance, and personal loss while striving to prove himself.
        """,
    },
    {
        "movie": "Spider-Man",
        "answer": """
        I know this:
        Director: Sam Raimi
        Cast: Tobey Maguire (Peter Parker/Spider-Man), Kirsten Dunst (Mary Jane Watson), Willem Dafoe (Norman Osborn/Green Goblin), James Franco (Harry Osborn)
        Budget: $139 million
        Box office revenue: $825 million
        Genre: Action, Superhero, Adventure
        Brief Synopsis: After being bitten by a genetically modified spider, high school student Peter Parker gains superpowers and learns the responsibility of being Spider-Man while facing his first major villain, the Green Goblin.
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {movie}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie": "Jurassic world"})


# with get_openai_callback() as usage:
#     a = chat.predict("What is the recipe for soju")
#     b = chat.predict("What is the recipe for bread")
#     print(a, "\n")
#     print(b, "\n")
#     print(usage)

AIMessage(content='\n        I know this:\n        Director: Colin Trevorrow\n        Cast: Chris Pratt (Owen Grady), Bryce Dallas Howard (Claire Dearing), Vincent D\'Onofrio (Vic Hoskins), Ty Simpkins (Gray Mitchell), Nick Robinson (Zach Mitchell)\n        Budget: $150 million\n        Box office revenue: $1.672 billion\n        Genre: Action, Adventure, Science Fiction\n        Brief Synopsis: Set 22 years after the events of the original "Jurassic Park," "Jurassic World" follows the reopening of the dinosaur theme park on Isla Nublar, where a genetically modified hybrid dinosaur escapes and wreaks havoc, leading to a fight for survival.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 147, 'prompt_tokens': 437, 'total_tokens': 584, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt

In [5]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is Nico")

/var/folders/q5/m8wmkz_d1c11pbrck3p2lm_m0000gn/T/ipykernel_94214/1596113238.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(
/var/folders/q5/m8wmkz_d1c11pbrck3p2lm_m0000gn/T/ipykernel_94214/1596113238.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(




> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is Nico
    You:


> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [6]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Nice to meet you, Nico! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


"That's great to know! How can I assist you with information or anything related to Seoul?"

In [7]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Nice to meet you, Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great to know! How can I assist you with information or anything related to Seoul?
    Human:What is my name?
    You:


> Finished chain.


'Your name is Nico.'